In [ ]:
env_DROPBOX_URL = ""
env_DROPBOX_TOKEN = ""
env_DROPBOX_KEY = ""
env_DROPBOX_SECRET = ""

env_CONTENT_PATH = ""
import sys
from settings import load_BatGutsSettings
S = load_BatGutsSettings()
env_CONTENT_PATH = S.zContentPath
env_DROPBOX_TOKEN = S.zDropboxToken
env_DROPBOX_URL = S.zDropboxUrl
env_DROPBOX_KEY = S.zDropboxKey
env_DROPBOX_SECRET = S.zDropboxSecret
if len(env_CONTENT_PATH) == 0:
    sys.exit("Exiting the script because setings were not processed")

In [ ]:
# Prompt the user for input
folder_path = input("Please enter the source dropbox subdirectory: ").strip()

# Print the input back to the user
print("You entered a folder path of:", folder_path)

In [ ]:
import os
from dropbox import DropboxOAuth2FlowNoRedirect
from dropbox.exceptions import AuthError
import requests

# Your app's key and secret
APP_KEY = env_DROPBOX_KEY
APP_SECRET = env_DROPBOX_SECRET

# Your refresh token
REFRESH_TOKEN =  env_DROPBOX_TOKEN

import requests

def refresh_access_token(app_key, app_secret, refresh_token):
    try:
        # Endpoint for refreshing the access token
        url = 'https://api.dropboxapi.com/oauth2/token'
        
        # Data for the POST request
        data = {
            'grant_type': 'refresh_token',
            'refresh_token': refresh_token,
            'client_id': app_key,
            'client_secret': app_secret
        }
        
        # Make the POST request
        response = requests.post(url, data=data)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            token_info = response.json()
            new_access_token = token_info['access_token']
            print("Access token refreshed successfully!")
            return new_access_token
        else:
            print(f"Error refreshing token. Status code: {response.status_code}")
            print(f"Response: {response.text}")
            return None
    
    except Exception as e:
        print(f"Error refreshing access token: {str(e)}")
        return None

# Refresh the access token
new_access_token = refresh_access_token(APP_KEY, APP_SECRET, REFRESH_TOKEN)

if new_access_token:
    print(f"New Access Token: {new_access_token}")
    
    # Optional: Test the new token
    try:
        headers = {
            "Authorization": f"Bearer {new_access_token}",
            "Content-Type": "application/json"
        }
        response = requests.post("https://api.dropboxapi.com/2/users/get_current_account", headers=headers)
        if response.status_code == 200:
            print("Token verified successfully!")
        else:
            print(f"Token verification failed. Status code: {response.status_code}")
    except Exception as e:
        print(f"Error testing token: {str(e)}")
else:
    print("Failed to refresh access token.")

In [ ]:

# Your app's key and secret
APP_KEY = env_DROPBOX_KEY
APP_SECRET = env_DROPBOX_SECRET

# Use your generated access token here
ACCESS_TOKEN=env_DROPBOX_TOKEN
import dropbox
from dropbox import Dropbox, files
from dropbox.exceptions import AuthError, BadInputError

# Use your generated access token here
SHARED_LINK = env_DROPBOX_URL

import requests

# Your app's key and secret
APP_KEY = 'YOUR_APP_KEY'
APP_SECRET = 'YOUR_APP_SECRET'

# Your refresh token
REFRESH_TOKEN = 'YOUR_REFRESH_TOKEN'

def refresh_access_token(app_key, app_secret, refresh_token):
    try:
        # Endpoint for refreshing the access token
        url = 'https://api.dropboxapi.com/oauth2/token'
        
        # Data for the POST request
        data = {
            'grant_type': 'refresh_token',
            'refresh_token': refresh_token,
            'client_id': app_key,
            'client_secret': app_secret
        }
        
        # Make the POST request
        response = requests.post(url, data=data)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            token_info = response.json()
            new_access_token = token_info['access_token']
            print("Access token refreshed successfully!")
            return new_access_token
        else:
            print(f"Error refreshing token. Status code: {response.status_code}")
            print(f"Response: {response.text}")
            return None
    
    except Exception as e:
        print(f"Error refreshing access token: {str(e)}")
        return None

# Refresh the access token
new_access_token = refresh_access_token(APP_KEY, APP_SECRET, REFRESH_TOKEN)

if new_access_token:
    print(f"New Access Token: {new_access_token}")
    
    # Optional: Test the new token
    try:
        headers = {
            "Authorization": f"Bearer {new_access_token}",
            "Content-Type": "application/json"
        }
        response = requests.post("https://api.dropboxapi.com/2/users/get_current_account", headers=headers)
        if response.status_code == 200:
            print("Token verified successfully!")
        else:
            print(f"Token verification failed. Status code: {response.status_code}")
    except Exception as e:
        print(f"Error testing token: {str(e)}")
else:
    print("Failed to refresh access token.")

In [ ]:
def get_size_of_folder(dbx, path):
    total_size = 0
    try:
        response = dbx.files_list_folder(path)
        for entry in response.entries:
            if isinstance(entry, dropbox.files.FileMetadata):
                total_size += entry.size
            elif isinstance(entry, dropbox.files.FolderMetadata):
                total_size += get_size_of_folder(dbx, entry.path_lower)
    except dropbox.exceptions.ApiError as err:
        print(f"Folder listing failed for {path}: {err}")
    return total_size

def get_shared_link_metadata(dbx, shared_link):
    try:
        link_metadata = dbx.sharing_get_shared_link_metadata(url=shared_link)
        return link_metadata
    except dropbox.exceptions.ApiError as err:
        print(f"Error retrieving shared link metadata: {err}")
        return None
    
dbx = Dropbox(ACCESS_TOKEN)
size = get_size_of_folder( dbx, "bat_guts")
print("The size of bat_guts is " + str(size))
